In [1]:
import pandas as pd
import nltk
import string
import re
import h2o
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html

In [2]:
tmp_df = pd.read_csv('D:/marco/vr_chat-master/Marco/train_yn.csv')
tmp_df.head()

,passages,query,query_id,query_type,answers_yn
0,"[Row(is_selected=0, passage_text=u'In his youn...",was ronald reagan a democrat,19700,description,Yes
1,"[Row(is_selected=0, passage_text=u'So, in the ...",do game shows pay their contestants,19749,description,Yes
2,"[Row(is_selected=0, passage_text=u'Even though...",does peanut butter contain cholesterol,19761,description,No
3,"[Row(is_selected=0, passage_text=u'When turnin...",can you take left hand lane to turn right on a...,19762,description,No
4,"[Row(is_selected=0, passage_text=u'Assault on ...",is armed assault a felony,19774,description,Yes


In [3]:
from nltk.corpus import stopwords 

stop_words = set(stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.porter import PorterStemmer

from nltk.stem.porter import *


In [4]:
def tf_cln(sent = tmp_df.iloc[0][0],cln=True):

    # sent = tmp_df.iloc[0][0]
    
    lowers= sent.lower()
    # print lower
    no_punctuation = lowers.translate(None, string.punctuation)
    slt_sent = re.findall(r'passagetext(.*?)url',no_punctuation)
    
    if(cln==True):
        slt_sent = re.findall(r'passage_text(.*?)url',sent)
#         print slt_sent
    else:
        slt_sent = sent
        
    token = nltk.word_tokenize(str(slt_sent))
    filtered_sentence = [w for w in token if not w in stop_words]
#     print filtered_sentence

    for i in filtered_sentence:
        if re.findall(r'^\W.*',str(i)):
    #         print re.findall(r'^\W',i)
            stop_words.add(i)
        
#     for i in re.findall(r'\W',str(filtered_sentence)):
#         stop_words.add(i)
#         print i

    stop_words.add('``')
    filtered_sentence = [w for w in token if not w in stop_words]

    tf = nltk.FreqDist(filtered_sentence)
    
    return dict(tf)

In [5]:
def get_tf_idf(dic):
    transformer = TfidfTransformer(smooth_idf=False)
    tf_idf=transformer.fit_transform(dic.values())
    return tf_idf.toarray().tolist()

In [6]:
lst_tfidf_psg=[]
for i in tmp_df.iloc[:,0]:
    lst_tfidf_psg.append(get_tf_idf(tf_cln(i)))

In [7]:
lst_tfidf_qry = []
for i in tmp_df.iloc[:,1]:
    lst_tfidf_qry.append(get_tf_idf(tf_cln(i,False)))

In [8]:
tmp_df = tmp_df.replace(tmp_df['query_type'].unique(),range(0,len(tmp_df['query_type'].unique())))
tmp_df['passages'] = lst_tfidf_psg
tmp_df['query'] = lst_tfidf_qry
tmp_df = tmp_df.replace(['yes', 'no'], ['Yes', 'No'])
tmp_df = tmp_df.replace(['Yes', 'No'], ['1', '0'])

In [9]:
del(tmp_df['query_id'])

In [10]:
tmp_df.head()

,passages,query,query_type,answers_yn
0,"[[0.034523833988, 0.069047667976, 0.0345238339...","[[0.57735026919, 0.57735026919, 0.57735026919]]",0,1
1,"[[0.0283410100633, 0.510138181139, 0.170046060...","[[0.5, 0.5, 0.5, 0.5]]",0,1
2,"[[0.0436020720197, 0.0872041440394, 0.08720414...","[[0.5, 0.5, 0.5, 0.5]]",0,0
3,"[[0.025515518154, 0.025515518154, 0.0637887953...","[[0.333333333333, 0.333333333333, 0.3333333333...",0,0
4,"[[0.0286534127531, 0.0286534127531, 0.02865341...","[[0.57735026919, 0.57735026919, 0.57735026919]]",0,1


In [106]:
tmp_df.iloc[0,0].tolist()

[[0.034523833987986266,
  0.06904766797597253,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.06904766797597253,
  0.034523833987986266,
  0.034523833987986266,
  0.06904766797597253,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.10357150196395881,
  0.10357150196395881,
  0.31071450589187644,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.06904766797597253,
  0.034523833987986266,
  0.034523833987986266,
  0.06904766797597253,
  0.10357150196395881,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.20714300392791762,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  0.034523833987986266,
  

In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,08 secs
H2O cluster version:,3.10.0.10
H2O cluster version age:,2 months and 10 days
H2O cluster name:,H2O_from_python_user_e0xgs4
H2O cluster total nodes:,1
H2O cluster free memory:,896 Mb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [94]:
h2o.H2OFrame?

In [13]:
hdf_tf=h2o.H2OFrame(tmp_df[tmp_df.columns[0:3]])
hdf_target = h2o.H2OFrame(pd.DataFrame(tmp_df.answers_yn))
hdf = h2o.H2OFrame.cbind(hdf_tf,h2o.H2OFrame.asfactor(hdf_target))

ValueError: `python_obj` is not a list of flat lists!

In [93]:
hdf.head(1)

NameError: name 'hdf' is not defined

In [ ]:
hdf_train, hdf_validate=hdf.split_frame(ratios=[0.75], seed =5010)

In [ ]:
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator